[Let's talk about LLM evaluation](https://huggingface.co/blog/clefourrier/llm-evaluation)

In [1]:
!pip install human-eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 876.5 kB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=2702720f717e745eb16360c32a6c1c248d9e709be119720b5addad787e9d7e5b
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
import torch
from transformers import AutoTokenizer,pipeline



# model_id = 'codellama/CodeLlama-13b-Instruct-hf'
model_id = 'codellama/CodeLlama-7b-Python-hf'


tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    #model_kwargs={"load_in_8bit": load_in_8bit}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
def generate_one_completion(message):
#     message = new_problems[_key]['prompt']
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        temperature=0.7,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

#     for seq in sequences:
#         print(f"Result: {seq['generated_text']}")
#     print("************")
    res = sequences[0]['generated_text']
    res = res.split(message)[-1]
#     print(res)
    return res

In [5]:
from human_eval.data import write_jsonl, read_problems

problems = read_problems()

num_samples_per_task = 1 #200
samples = [
    dict(task_id=task_id, completion=generate_one_completion(problems[task_id]["prompt"]))
    for task_id in problems
    for _ in range(num_samples_per_task)
]
write_jsonl("samples.jsonl", samples)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting

In [6]:
!cat samples.jsonl

{"task_id": "HumanEval/0", "completion": "    for i in range(len(numbers)):\n        for j in range(len(numbers)):\n            if abs(numbers[i] - numbers[j]) <= threshold and i!= j:\n                return True\n    return False\n\n\nif __name__ == \"__main__\":\n    import doctest\n\n    doctest.testmod()\n"}
{"task_id": "HumanEval/1", "completion": "    if paren_string == '':\n        return []\n    if paren_string == '(':\n        return ['(']\n\n    # find first open brace\n    left_brace_index = paren_string.index('(')\n    # find first close brace that is not nested within any group\n    right_brace_index = paren_string[left_brace_index:].index(')') + left_brace_index\n\n    if right_brace_index < len(paren_string) - 1:\n        return [paren_string[:right_brace_index + 1]] + separate_paren_groups(paren_string[right_brace_index + 1:])\n    return [paren_string]\n\n\ndef get_paren_groups(paren_string: str) -> List[str]:\n    \"\"\" Input to this function is a string containing m

In [7]:
!evaluate_functional_correctness samples.jsonl

Reading samples...
164it [00:00, 33180.54it/s]
Running test suites...
100% 164/164 [00:04<00:00, 33.90it/s]
Writing results to samples.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 23033.28it/s]
{'pass@1': 0.3231707317073171}
